In [1]:
# This is a generic bot carry out to perform Two operation:
# Weather
# Corona Virus updates.

In [11]:
import random
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
nltk.download('punkt')
import numpy as np
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
def search_user_response(res):
    try: 
        from googlesearch import search 
    except ImportError:  
        print("No module named 'google' found") 
    #query = ''.join(res.split(' '))  
    for j in search(res, tld="co.in", num=10, stop=1, pause=2): 
        return (j) 


In [13]:
import requests,re,random
from bs4 import BeautifulSoup
def get_weather_data(res):
    weather_inputs = ['temperature','temp','rain','weather','sunny']
    for inputs in res.split():
        if inputs in weather_inputs:            
            url = search_user_response(weather_inputs[3])
            page = requests.get(url,verify=False)
            weather=[]
            if page.status_code==200:
                soup = BeautifulSoup(page.content, 'html.parser')
                for div in soup.find_all('div', attrs={'class': re.compile("^today_nowcard-phrase")}):
                    weather.append(div.text)
                for div in soup.find_all('div', attrs={'class': re.compile("^today_nowcard-feels")}):
                    weather.append(div.text)
                #weather.append('Weather is'+weather[0]+' '+weather[1])
                try:
                    return(random.choice(weather))
                except IndexError:
                    return('Please visit '+url+' for more info')





In [5]:
def get_corona_data(res):
    inputs = ['corona','virus','prevent','symptoms','disease']
    corpus =[]
    for inp in res.split():
        if inp in inputs:
            for i in range(1,4):
                url = 'https://www.who.int/health-topics/coronavirus#tab=tab_'+str(i)
                page = requests.get(url,verify=False)
                if page.status_code==200:
                    soup = BeautifulSoup(page.content, 'html.parser')
                    for p in soup.find_all('p'):
                        corpus.append(p.get_text())
            url_myth = 'https://www.who.int/emergencies/diseases/novel-coronavirus-2019/advice-for-public/myth-busters'
            page = requests.get(url,verify=False)
            soup = BeautifulSoup(page.content, 'html.parser')
            for p in soup.find_all('p'):
                corpus.append(p.get_text())
                    
    text =''
    #print(corpus)
    for ele in corpus:
        text+=ele
    #print('text:',text)
    sent_tokens = nltk.sent_tokenize(text) #Convert the text into a list of sentences

#Print the list of sentences
    #print(sent_tokens)
    
    return sent_tokens

In [14]:
#Tokenization

#Create a dictionary (key:value) pair to remove punctuations
remove_punct_dict = dict(  ( ord(punct),None) for punct in string.punctuation)

#Print the punctuations
print(string.punctuation)

#Print the dictionary
print(remove_punct_dict)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
{33: None, 34: None, 35: None, 36: None, 37: None, 38: None, 39: None, 40: None, 41: None, 42: None, 43: None, 44: None, 45: None, 46: None, 47: None, 58: None, 59: None, 60: None, 61: None, 62: None, 63: None, 64: None, 91: None, 92: None, 93: None, 94: None, 95: None, 96: None, 123: None, 124: None, 125: None, 126: None}


In [ ]:
def LemNormalize(text):
  return nltk.word_tokenize(text.lower().translate(remove_punct_dict))

#Print the tokenization text
print(LemNormalize(text))

In [8]:
#Keyword Matching

#Greeting Inputs
GREETING_INPUTS = ["hi", "hello", "hola", "greetings", "wassup", "hey"]

#Greeting responses back to the user
GREETING_RESPONSES=["howdy", "hi", "hey", "what's good", "hello", "hey there"]

#Function to return a random greeting response to a users greeting
def greeting(sentence):
  #if the user's input is a greeting, then return a randomly chosen greeting response
  for word in sentence.split():
    if word.lower() in GREETING_INPUTS:
        return random.choice(GREETING_RESPONSES)

In [15]:
def response(user_response):
  

  #The users response / query
                
  user_response = user_response.lower() #Make the response lower case

  ###Print the users query/ response
  #print(user_response)

  #Set the chatbot response to an empty string
  robo_response = ''
  sent_tokens = get_corona_data(user_response)

  #Append the users response to the sentence list
  sent_tokens.append(user_response)

  ###Print the sentence list after appending the users response
  #print(sent_tokens)

  #Create a TfidfVectorizer Object
  TfidfVec = TfidfVectorizer(tokenizer = LemNormalize, stop_words='english')

  #Convert the text to a matrix of TF-IDF features
  tfidf = TfidfVec.fit_transform(sent_tokens)

  ###Print the TFIDF features
  #print(tfidf)
  #Get the measure of similarity (similarity scores)
  vals = cosine_similarity(tfidf[-1], tfidf)

  #Print the similarity scores
  #print(vals)

  #Get the index of the most similar text/sentence to the users response
  try:
        idx = vals.argsort()[0][-2]
  except IndexError :
        robo_response = robo_response+"I apologize, I don't understand."
        return robo_response

  #Reduce the dimensionality of vals
  flat = vals.flatten()

  #sort the list in ascending order
  flat.sort()

  #Get the most similar score to the users response
  score = flat[-2]

  #Print the similarity score
  #print(score)

  #If the variable 'score' is 0 then their is no text similar to the users response
  if(score == 0):
    robo_response = robo_response+"I apologize, I don't understand."
  else:
    robo_response = robo_response+sent_tokens[idx]
  
  #Print the chat bot response
  #print(robo_response)
  
  #Remove the users response from the sentence tokens list
  sent_tokens.remove(user_response)
  
  return robo_response


In [16]:
flag = True
print("jedi: I will answer your queries about Weather and Corona Virus. If you want to exit, type Bye!")
while(flag == True):
  user_response = input()
  user_response = user_response.lower()
  if(user_response != 'bye'):
    if(user_response == 'thanks' or user_response =='thank you'):
      flag=False
      print("jedi: You are welcome !")
    else:
      if(greeting(user_response) != None):
        print("jedi: "+greeting(user_response))
      elif(get_weather_data(user_response)!= None):
        print("jedi: "+get_weather_data(user_response))
#       elif(get_corona_data(user_response)!= None): 
#         print("jedi: "+get_corona_data(user_response))
      else:
        print("jedi: "+response(user_response))       
  else:
    flag = False
    print("jedi: Chat with you later !")

jedi: I will answer your queries about Weather and Corona Virus. If you want to exit, type Bye!
Hi
jedi: howdy
Hello
jedi: what's good
whats weather like today
jedi: Feels Like 24°
what is corona virus symptoms
jedi: People who have underlying medical conditions and those over 60 years old have a higher risk of developing severe disease and death.Common symptoms include:Other symptoms include:People with mild symptoms who are otherwise healthy should self-isolate and contact their medical provider or a COVID-19 information line for advice on testing and referral.People with fever, cough or difficulty breathing should call their doctor and seek medical attention.
what is corona virus ?
jedi: Coronavirus disease (COVID-19) is an infectious disease caused by a newly discovered coronavirus.Most people infected with the COVID-19 virus will experience mild to moderate respiratory illness and recover without requiring special treatment.
prevention from corona virus ?
jedi: Coronavirus disease